In [139]:
import re
inspeech = {}
vbind = {'i':0, 'm': 1, 'p':2, 's':3, 'o' : 4, 'n': 5, '1':6,'tot': 7}


f = open("/Users/gcrane/github/Thucydides-new-working-materials/thuc-speeches.tsv","r")
for l in f:
    l = re.sub('\s+$','',l)
    m = re.search('([1-8]|[1-8]\.[0-9]+)\.([0-9]+)\-([1-8]|[1-8]\.[0-9]+)\.([0-9]+)$',l)
    if(m):
        book = m[1]
        start = m[2]
        stop = m[4]
        #print(book,start,stop,l)

        args = l.split('\t')
        speaker = args[0]
        location = args[1]
        if( int(stop) < int(start) ):
            print("error",stop,"less than", start,l)
        for x in range(int(start),int(stop)):
            inspeech[str(book)+"."+str(x)] = speaker + '\t' + location
        inspeech[str(book)+"."+stop] = speaker + '\t' + location

def gettreebank(tbname):
    global vbind
    f = open(tbname)
    curcit = ''
    curtbank = {}
    cittab = {}
    senttab = {}
    sentverbs = {}
    sentcits = {}
    senttexts = {}
    
    
    for l in f:
        l = re.sub('\s+$','',l)
        
        
        m = re.search('subdoc="([^"]+)"',l)
        if(m):
            curcit = m[1]
            
        m = re.search('form="([^"]+)"',l)
        if(m):
            curform = m[1]
            
        m = re.search('sentence id="([^"]+)"',l)
        if(m):
            sentid = m[1]
            sentverbs[sentid] = [0,0,0,0,0,0,0,0]
            sentcits[sentid] = curcit
            senttexts[sentid] = ''
            
        m = re.search('word id="([^"]+)"',l)
        if(m):
            curid = m[1]
            
            curtbank[curid] = l
            cittab[curid] = curcit
            senttab[curid] = sentid
            
            sentverbs[sentid][vbind['tot']] = sentverbs[sentid][vbind['tot']] + 1
            if(senttexts[sentid]):
                senttexts[sentid] = senttexts[sentid] + ' '
            senttexts[sentid] = senttexts[sentid] + curform
        
        m = re.search('postag="v(1)',l)
        if(m):
            curv = vbind[m[1]]
            sentverbs[sentid][curv] = sentverbs[sentid][curv] + 1
            
        m = re.search('postag="v...(.)',l)
        if(m):
            curv = vbind[m[1]]
            sentverbs[sentid][curv] = sentverbs[sentid][curv] + 1
    return curtbank,sentcits,sentverbs,senttexts
            
            
thuctb,thuccits,thucsents,thuctexts = gettreebank('0003-001.xml')

outf = open('thucsentvbs.tsv','w')

print('cit','gen/spk','loc','sentid','ind-noninds','\t'.join(vbind),'txt',sep='\t',file=outf)
narrs = 0
speeches = 0
narr_ind2non = 0
speech_ind2non = 0
for foo in thuctexts:
    nonind = 0
    noninds = str(thucsents[foo][0]) + '\t'
    for i in range(1,7):
        noninds = noninds + str(thucsents[foo][i]) + '\t'
        nonind = thucsents[foo][i] + nonind
    noninds = noninds + str(thucsents[foo][7]) + '\t' + str(thucsents[foo][6])
    bookchapsection = thuccits[foo]
    bookchap = re.sub('\.[0-9]+$','',thuccits[foo])
    usecit = bookchap
    indnonind = thucsents[foo][0]-nonind
    
    if(bookchap in inspeech):
        genre = inspeech[bookchap]
        speeches = speeches + 1
        speech_ind2non = speech_ind2non + indnonind
    elif( bookchapsection in inspeech):
        genre = inspeech[bookchapsection]
        speeches = speeches + 1
        usecit = bookchapsection
        speech_ind2non = speech_ind2non + indnonind

        
    else:
        
        genre = 'narrative\ttbd'
        narrs = narrs + 1
        narr_ind2non = narr_ind2non + indnonind
    #if(thucsents[foo][0] and nonind == 0):
    if(thucsents[foo][6] and not usecit in inspeech):
        print(thuccits[foo],genre,foo,thucsents[foo][0],noninds,thuctexts[foo],sep='\t')
        
    print(thuccits[foo],genre,foo,indnonind,thucsents[foo][0],noninds,thuctexts[foo],sep='\t',file=outf)
        
outf.close()
print('speeches',speeches,speech_ind2non/speeches)
print('narratives',narrs,narr_ind2non/narrs)
            

speeches 1450 -2.04551724137931
narratives 4605 -0.6777415852334419


In [132]:
'\t'.join(vbind)

'i\tm\tp\ts\to\tn\t1\ttot'

In [87]:
foo = [0,1,3]
print('\t'.join(foo))



TypeError: sequence item 0: expected str instance, int found